# Fundamentação Metodológica

Este notebook implementa as **três primeiras etapas do processo KDD** (Knowledge Discovery in Databases):

1. **Seleção**: Identificação e carregamento dos dados relevantes
2. **Pré-processamento**: Limpeza e tratamento de inconsistências
3. **Transformação**: Criação de atributos derivados (flags e métricas)

**Referências:**
- Fayyad, Piatetsky-Shapiro e Smyth (1996): Definição do processo KDD
- Galvão e Marin (2009): Técnicas de preparação de dados para mineração

### Justificativa das Flags Criadas

As flags binárias criadas neste notebook são fundamentadas em:

1. **Lei 14.133/2021 (Art. 23, 34, 59)**: Critérios de julgamento e valores de referência
2. **Torres-Berru e Batista (2021)**: Parâmetros de risco em licitações
3. **Carvalho e Filho (2024)**: Abordagem analítica para detecção de anomalias

**Flags implementadas:**
- `flag_homologado_acima_estimado`: Identifica sobrepreço (Art. 23, Lei 14.133)
- `flag_sem_desconto`: Ausência de economia (indicador de competitividade)
- `flag_nao_menor_preco`: Critério diferente de menor preço
- `flag_certame_em_andamento`: Processo não finalizado
- `flag_informatica`: Filtro temático para o nicho de TI


## 1 - Seleção de Dados

**Objetivo.** Definir quais arquivos CSV participarão do ciclo de pré-processamento. Cada arquivo contém colunas com o mesmo layout (separador `;`, cabeçalho padrão do portal de licitações).

**Variáveis principais.**
- `csv_paths`: lista com os caminhos de cada ano (2022–2025). É possível incluir novos anos adicionando linhas à lista.
- `frames`: coleção temporária para armazenar cada tabela antes da concatenação.

**Resultado esperado.** Um `DataFrame` denominado `raw_df` com todos os registros.


In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from IPython.display import display, FileLink

pd.options.display.max_colwidth = 120
pd.options.display.float_format = lambda x: f"{x:,.2f}"

In [2]:
# Conjunto de dados

csv_paths = [
    Path('./data/TB_LICITACOES-2025.csv'),
    Path('./data/TB_LICITACOES-2024.csv'),
    Path('./data/TB_LICITACOES-2023.csv'),
    Path('./data/TB_LICITACOES-2022.csv'),
]

frames = []
for path_csv in csv_paths:
    frame = (
        pd.read_csv(path_csv, sep=';', dtype=str)
            .rename(columns=str.lower)
            .assign(arquivo=path_csv.name)
    )
    frames.append(frame)

raw_df = pd.concat(frames, ignore_index=True)
print(f'Registros carregados: {len(raw_df):,}')
raw_df.head()


Registros carregados: 9,810


,numprocesso,ano,modalidade,situacao,objeto,orgao,registrodepreco,vigenciaata,revogacaoata,criteriojulgamento,valor_estimado,valor_ou_desconto_homologado,dataabertura,datahomologacao,localdisputa,idlicitacoes,protocolo,tempoprazocontrato,arquivo
0,1,2025,Audiência Pública,Fase Certame,Audiência Pública visa consultar os eventuais interessados no Credenciamento de Entidades Médicas e Psicológicas de ...,DETRAN-Departamento de Trânsito do Estado do Paraná,Não,NaN,NaN,-,0.00,NaN,2025-02-19,NaN,NaN,NaN,234022687,NaN,TB_LICITACOES-2025.csv
1,2,2025,Audiência Pública,Adjudicado,AUDIÊNCIA PÚBLICA DO PROGRAMA GANHANDO O MUNDO.,SEED-Secretaria de Estado da Educação,Não,NaN,NaN,Menor Preço,0.00,NaN,2025-05-22,NaN,NaN,NaN,239262546,1 Parcela(s) - Empreitada/Global,TB_LICITACOES-2025.csv
2,3,2025,Audiência Pública,Fase Certame,Audiência Pública que visa obter contribuições para compor o Estudo Técnico Preliminar e Termo de Referência para po...,SEAP-Secretaria de Estado da Administração e da Previdência,Não,NaN,NaN,Menor Preço,0.00,NaN,2025-06-04,NaN,NaN,NaN,239013830,NaN,TB_LICITACOES-2025.csv
3,4,2025,Audiência Pública,Fase Certame,Audiência Pública que visa subsidiar o processo de Contratação de Empresa Especializada para Prestação de Serviços d...,SEAP-Secretaria de Estado da Administração e da Previdência,Não,NaN,NaN,Menor Preço,0.00,NaN,2025-07-08,NaN,NaN,NaN,241710122,NaN,TB_LICITACOES-2025.csv
4,5,2025,Audiência Pública,Fase Certame,Audiência Pública que visa obter contribuições para compor o Estudo Técnico Preliminar e Termo de Referência para po...,SEAP-Secretaria de Estado da Administração e da Previdência,Não,NaN,NaN,Menor Preço,0.00,NaN,2025-07-10,NaN,NaN,NaN,241858502,NaN,TB_LICITACOES-2025.csv



## 2 - Pré-processamento &
## 3 - Tratamento dos Dados

Este bloco aplica correções de tipos, higieniza texto e cria as primeiras features numéricas.

**Passos executados pelo código a seguir.**
1. `to_datetime(series, fmt)`: converte strings para `datetime` usando a máscara `%Y-%m-%d`. Caso haja inconsistência, recebe `NaT` (not-a-time).
   \[
   	ext{data_convertida}_i = egin{cases}
       	ext{pd.to_datetime}(s_i, 	ext{format}=fmt) & 	ext{se válido} \
       	ext{NaT} & 	ext{caso contrário}
   \end{cases}
   \]
2. Conversão dos valores monetários para `float`. Primeiro removemos separadores, depois trocamos vírgula por ponto e aplicamos `pd.to_numeric` com tratamento de erro.
3. Normalização do texto descritivo `objeto`: `lower()`, `strip()` e colapso de múltiplos espaços com regex (`\s+ -> ' '`).
4. Criação das variáveis derivadas:
   - \(	ext{diferença\_valor}_i = 	ext{valor\_ou\_desconto\_homologado}_i - 	ext{valor\_estimado}_i\)
   - \(	ext{variação\_percentual}_i = 
rac{	ext{diferença\_valor}_i}{	ext{valor\_estimado}_i} 	imes 100\)
   - \(	ext{proporção\_homologado\_estimado}_i = 
rac{	ext{valor\_ou\_desconto\_homologado}_i}{	ext{valor\_estimado}_i}\)
   - \(	ext{lead\_time\_dias}_i = (	ext{data\_homologação}_i - 	ext{data\_abertura}_i)\)
5. Flags categóricas:
   - `flag_homologado_acima_estimado` (proporção > 1);
   - `flag_houve_desconto` (proporção ∈ [0, 1));
   - `criterio_menor_preco` e `flag_situacao_certame` a partir de buscas lexicais em colunas categóricas.


In [3]:

# Função auxiliar para padronizar colunas de data no formato datetime.
def to_datetime(series, fmt="%Y-%m-%d"):
    '''Converte strings de data para datetime, retornando `NaT` em caso de erro.'''
    return pd.to_datetime(series, format=fmt, errors='coerce')


clean_df = raw_df.copy()

# Colunas descartadas por não agregarem valor ao modelo inicial.
drop_columns = ['registrodepreco', 'vigenciaata', 'revogacaoata', 'localdisputa', 'idlicitacoes', 'protocolo', 'tempoprazocontrato']

clean_df = clean_df.drop(columns=drop_columns, errors='ignore')
clean_df['dataabertura'] = to_datetime(clean_df['dataabertura'])
clean_df['datahomologacao'] = to_datetime(clean_df['datahomologacao'])

for col in ['valor_estimado', 'valor_ou_desconto_homologado']:
    clean_df[col] = (
        clean_df[col]
        .str.replace('.', '', regex=False)
        .str.replace(',', '.', regex=False)
    )
    clean_df[col] = pd.to_numeric(clean_df[col], errors='coerce')

clean_df['objeto_normalizado'] = clean_df['objeto'].str.lower().str.strip()
clean_df['objeto_normalizado'] = clean_df['objeto_normalizado'].str.replace(r"\s+", " ", regex=True)

# Engenharia de atributos para enriquecer análises futuras.
possui_valores = (
    clean_df['valor_estimado'].notna()
    & clean_df['valor_estimado'].ne(0)
    & clean_df['valor_ou_desconto_homologado'].notna()
)


# ===========================================================
# FLAGS
# ===========================================================


# Gap financeiro absoluto; destaca sobrepreço ou desconto frente ao estimado.
clean_df['diferenca_valor'] = np.where(
    possui_valores,
    clean_df['valor_ou_desconto_homologado'] - clean_df['valor_estimado'],
    np.nan
)

# Gap relativo em porcentagem; facilita comparar licitações de tamanhos diferentes.
clean_df['variacao_percentual'] = np.where(
    possui_valores,
    (clean_df['diferenca_valor'] / clean_df['valor_estimado']) * 100,
    np.nan
)

# Relação direta entre valores homologado/estimado; usada em flags binárias.
clean_df['proporcao_homologado_estimado'] = np.where(
    possui_valores,
    clean_df['valor_ou_desconto_homologado'] / clean_df['valor_estimado'],
    np.nan
)

# Flags que indicam desconto ou sobrepreço para o classificador.
clean_df['flag_homologado_acima_estimado'] = clean_df['proporcao_homologado_estimado'] > 1
clean_df['flag_houve_desconto'] = clean_df['proporcao_homologado_estimado'].between(0, 1, inclusive='left')

# Tempo entre abertura e homologação; sinaliza agilidade ou atrasos do processo.
clean_df['lead_time_dias'] = (
    clean_df['datahomologacao'] - clean_df['dataabertura']
).dt.days

# Flags com sinais de modalidade/etapa.
clean_df['criterio_menor_preco'] = clean_df['criteriojulgamento'].str.contains('Menor Preço', case=False, na=False)
clean_df['flag_situacao_certame'] = clean_df['situacao'].str.contains('Fase Certame', case=False, na=False)


# ===========================================================
# TRATAMENTO
# ===========================================================


# Ajustes finais das variáveis derivadas antes da mineração de dados.

cond_valores = (
    clean_df['valor_estimado'].notna()
    & clean_df['valor_estimado'].ne(0)
    & clean_df['valor_ou_desconto_homologado'].notna()
)

# 1. Diferenca e proporção: aplica ordem desejada e trata ausências com zero.
clean_df['diferenca_valor'] = np.where(
    clean_df['valor_estimado'].notna() & clean_df['valor_ou_desconto_homologado'].notna(),
    clean_df['valor_estimado'] - clean_df['valor_ou_desconto_homologado'],
    0.0
)

clean_df['variacao_percentual'] = np.where(
    cond_valores,
    (clean_df['diferenca_valor'] / clean_df['valor_estimado']) * 100,
    0.0
)

clean_df['proporcao_homologado_estimado'] = np.where(
    cond_valores,
    clean_df['valor_ou_desconto_homologado'] / clean_df['valor_estimado'],
    0.0
)

# Flags de desconto/sobrepreço se atualizam automaticamente porque dependem da proporção.
clean_df['flag_homologado_acima_estimado'] = clean_df['proporcao_homologado_estimado'] > 1
clean_df['flag_houve_desconto'] = clean_df['proporcao_homologado_estimado'].between(0, 1, inclusive='left')

# 2. Lead time negativo não faz sentido cronológico; forçamos limite inferior 0 e tratamos ausentes com 0.
clean_df['lead_time_dias'] = clean_df['lead_time_dias'].clip(lower=0).fillna(0)




print(f'Registros carregados: {len(raw_df):,}')
clean_df.head()


Registros carregados: 9,810


,numprocesso,ano,modalidade,situacao,objeto,orgao,criteriojulgamento,valor_estimado,valor_ou_desconto_homologado,dataabertura,...,arquivo,objeto_normalizado,diferenca_valor,variacao_percentual,proporcao_homologado_estimado,flag_homologado_acima_estimado,flag_houve_desconto,lead_time_dias,criterio_menor_preco,flag_situacao_certame
0,1,2025,Audiência Pública,Fase Certame,Audiência Pública visa consultar os eventuais interessados no Credenciamento de Entidades Médicas e Psicológicas de ...,DETRAN-Departamento de Trânsito do Estado do Paraná,-,0.00,NaN,2025-02-19,...,TB_LICITACOES-2025.csv,audiência pública visa consultar os eventuais interessados no credenciamento de entidades médicas e psicológicas de ...,0.00,0.00,0.00,False,True,0.00,False,True
1,2,2025,Audiência Pública,Adjudicado,AUDIÊNCIA PÚBLICA DO PROGRAMA GANHANDO O MUNDO.,SEED-Secretaria de Estado da Educação,Menor Preço,0.00,NaN,2025-05-22,...,TB_LICITACOES-2025.csv,audiência pública do programa ganhando o mundo.,0.00,0.00,0.00,False,True,0.00,True,False
2,3,2025,Audiência Pública,Fase Certame,Audiência Pública que visa obter contribuições para compor o Estudo Técnico Preliminar e Termo de Referência para po...,SEAP-Secretaria de Estado da Administração e da Previdência,Menor Preço,0.00,NaN,2025-06-04,...,TB_LICITACOES-2025.csv,audiência pública que visa obter contribuições para compor o estudo técnico preliminar e termo de referência para po...,0.00,0.00,0.00,False,True,0.00,True,True
3,4,2025,Audiência Pública,Fase Certame,Audiência Pública que visa subsidiar o processo de Contratação de Empresa Especializada para Prestação de Serviços d...,SEAP-Secretaria de Estado da Administração e da Previdência,Menor Preço,0.00,NaN,2025-07-08,...,TB_LICITACOES-2025.csv,audiência pública que visa subsidiar o processo de contratação de empresa especializada para prestação de serviços d...,0.00,0.00,0.00,False,True,0.00,True,True
4,5,2025,Audiência Pública,Fase Certame,Audiência Pública que visa obter contribuições para compor o Estudo Técnico Preliminar e Termo de Referência para po...,SEAP-Secretaria de Estado da Administração e da Previdência,Menor Preço,0.00,NaN,2025-07-10,...,TB_LICITACOES-2025.csv,audiência pública que visa obter contribuições para compor o estudo técnico preliminar e termo de referência para po...,0.00,0.00,0.00,False,True,0.00,True,True



### Interpretação das variáveis derivadas

| Variável | Fórmula / Definição | Contexto analítico |
| --- | --- | --- |
| `diferenca_valor` | \(valor\_estimado - valor\_ou\_desconto\_homologado\) | Valores positivos representam economia; negativos sugerem sobrepreço. Ausências são tratadas como 0. |
| `variacao_percentual` | \(
rac{valor\_estimado - valor\_ou\_desconto\_homologado}{valor\_estimado} 	imes 100\) | Normaliza o gap para comparar licitações de portes diferentes. |
| `proporcao_homologado_estimado` | \(
rac{valor\_ou\_desconto\_homologado}{valor\_estimado}\) | Base para flags; >1 indica homologação acima do previsto. |
| `flag_homologado_acima_estimado` | `proporcao_homologado_estimado > 1` | Sinaliza possíveis riscos financeiros. |
| `flag_houve_desconto` | `0 ≤ proporcao_homologado_estimado < 1` | Identifica economia obtida no certame. |
| `lead_time_dias` | `max((datahomologacao - dataabertura).days, 0)` | Prazos longos podem indicar morosidade. |
| `criterio_menor_preco` | `True` se `criteriojulgamento` contém "Menor Preço" | Conecta com princípios da Lei nº 14.133/2021. |
| `flag_situacao_certame` | `True` se `situacao` contém "Fase Certame" | Sinaliza processos em andamento. |
| `flag_informatica` | Resultado do filtro por palavras-chave | Pseudo-rótulo do nicho de TI. |



### Filtro para licitações de informática

**Objetivo.** Criar um subconjunto temático alinhado ao escopo do TCC (informática, hardware, software, serviços de TI).

**Componentes.**
- `keyword_groups`: dicionário com categorias e termos relevantes.
- `mask`: função `lambda` que verifica se algum termo está presente em `objeto_normalizado`. A lógica equivale a:
  \[
  	ext{mask}_i = egin{cases}
     1 & 	ext{se } \exists 	ext{ palavra-chave } k 	ext{ tal que } k \in objeto\_normalizado_i \
     0 & 	ext{caso contrário}
  \end{cases}
  \]
- `flag_informatica`: coluna booleana inserida em `clean_df` para futura modelagem.

**Saída.** `informatica_df`, usado tanto para análises específicas quanto para exportação.


In [5]:

# Termos específicos para softwares, hardwares ou serviços especializados.

keyword_groups = {
    'software': ['software', 'licença', 'sistema', 'aplicativo'],
    'hardware': ['computador', 'notebook', 'servidor', 'periférico', 'roteador', 'celular', 'dispositivo'],
    'serviços': ['suporte técnico', 'manutenção de ti', 'serviços de ti', 'informática', 'videomonitoramento', 'integração de dados']
}

keywords = [kw for group in keyword_groups.values() for kw in group]

mask = clean_df['objeto_normalizado'].fillna('').apply(
    lambda text: any(kw in text for kw in keywords)
)

# Marca o dataset principal para reutilização em tarefas supervisionadas/não supervisionadas.
clean_df['flag_informatica'] = mask

informatica_df = clean_df[mask].copy()
print(f"Registros relacionados à informática: {len(informatica_df):,}")
informatica_df.head()


Registros relacionados à informática: 866


,numprocesso,ano,modalidade,situacao,objeto,orgao,criteriojulgamento,valor_estimado,valor_ou_desconto_homologado,dataabertura,...,objeto_normalizado,diferenca_valor,variacao_percentual,proporcao_homologado_estimado,flag_homologado_acima_estimado,flag_houve_desconto,lead_time_dias,criterio_menor_preco,flag_situacao_certame,flag_informatica
3,4,2025,Audiência Pública,Fase Certame,Audiência Pública que visa subsidiar o processo de Contratação de Empresa Especializada para Prestação de Serviços d...,SEAP-Secretaria de Estado da Administração e da Previdência,Menor Preço,0.00,NaN,2025-07-08,...,audiência pública que visa subsidiar o processo de contratação de empresa especializada para prestação de serviços d...,0.00,0.00,0.00,False,True,0.00,True,True,True
8,10,2025,Audiência Pública,Fase Certame,"relativo a contratação de empresa, ou consórcio de empresa para implantação da solução tecnológica com o objetivo fo...",SGSD-Superintendência Geral de Governança de Serviços e Dados,Menor Preço,0.00,NaN,2025-09-11,...,"relativo a contratação de empresa, ou consórcio de empresa para implantação da solução tecnológica com o objetivo fo...",0.00,0.00,0.00,False,True,0.00,True,True,True
30,23,2025,Chamamento Público,Fase Certame,"Credenciamento de Permissionários para exercer a permissão administrativa de uso do espaço público, a título gratuit...",DER-Departamento de Estradas de Rodagem do Estado do Paraná,-,0.00,NaN,2025-09-17,...,"credenciamento de permissionários para exercer a permissão administrativa de uso do espaço público, a título gratuit...",0.00,0.00,0.00,False,True,0.00,False,True,True
67,13,2025,Concorrência Eletrônica (Lei Federal 14.133/2021),Homologado,"Execução dos serviços para recuperação de aterro, contenção do processo erosivo e melhorias no sistema de drenagem e...",DER-Departamento de Estradas de Rodagem do Estado do Paraná,Menor Preço,"90,962,114.00","60,000,000.00",2025-04-15,...,"execução dos serviços para recuperação de aterro, contenção do processo erosivo e melhorias no sistema de drenagem e...","30,962,114.00",34.04,0.66,False,True,41.00,True,False,True
68,14,2025,Concorrência Eletrônica (Lei Federal 14.133/2021),Homologado,Execução de obras de contenção de águas pluviais e demais dispositivos de drenagem na rodovia PR-685 trecho Entr. PR...,DER-Departamento de Estradas de Rodagem do Estado do Paraná,Menor Preço,"95,414,521.00","56,999,000.00",2025-04-14,...,execução de obras de contenção de águas pluviais e demais dispositivos de drenagem na rodovia pr-685 trecho entr. pr...,"38,415,521.00",40.26,0.60,False,True,57.00,True,False,True



### Exportação dos resultados

Dois arquivos CSV são gerados na pasta `exports/`:
1. `licitacoes_preprocessadas.csv`: contém todo o dataset limpo e enriquecido.
2. `licitacoes_informatica.csv`: subconjunto filtrado pelo nicho de TI.

O código utiliza `Path.mkdir` (cria diretório se necessário) e `FileLink` para facilitar o download direto no notebook.


In [6]:
# Função utilitária para exportar DataFrames limpos e gerar links de download.
def exportar_para_csv(dataframe, filename):
    """Salva o DataFrame em `exports/filename` usando separador ';' e retorna o link."""
    destino = Path('exports')
    destino.mkdir(exist_ok=True)
    output_path = destino / filename
    dataframe.to_csv(output_path, sep=';', index=False)
    return FileLink(output_path)

link_dataset_limpo = exportar_para_csv(clean_df, '1_licitacoes_preprocessadas.csv')
link_dataset_informatica = exportar_para_csv(informatica_df, '2_licitacoes_informatica.csv')

display({'preprocessado': link_dataset_limpo, 'informatica': link_dataset_informatica})


{'preprocessado': /Users/azevedo/GitHub/anomalyClassifier/exports/1_licitacoes_preprocessadas.csv,
 'informatica': /Users/azevedo/GitHub/anomalyClassifier/exports/2_licitacoes_informatica.csv}